In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup as bs
import requests
import time
import re

In [ ]:
# Value placeholder
name_of_place = []
location = []
category = []
number_likes = []
description = []
url = []
base_url = "https://hotels.ng"
pages = 1

filename = "datafile/Sights_in_Nigeria.csv"
data = pd.read_csv(filename)

for page in range(365):
    page_url = f"https://hotels.ng/places/nigeria-1/{pages}"
    response = requests.get(page_url)
    # time.sleep(3)
    page_content = response.text
    soup_doc = bs(page_content, "html.parser")
    response.raise_for_status()


    print(f"Scraping data from page{pages}")
    # scraping names of places
    name_tags = soup_doc.select("a > h2")
    for i in name_tags:
        places = i.text.strip()
        name_of_place.append(places)

    # scraping the locations
    location_tags = soup_doc.find_all("div", {"class":"category-de01"})
    for loc in location_tags:
        location_p_tags = loc.find_all("p")
        locations = location_p_tags[1].text.strip()
        location.append(locations)

    #scraping the categories and description data
    categories_placeholder = soup_doc.findAll("div", {"class":"category-de01"})
    for categories in categories_placeholder:
        category_tags = categories.find_all("p")
        category_label = category_tags[0].text.strip()
        category.append(category_label)


    description_tag = soup_doc.find_all("p", {"class":"sub-details"})
    for desc in description_tag:
        descriptions = desc.text.strip()
        descriptions = descriptions.replace("\n", "")
        descriptions = descriptions.replace("\r", "")
        description.append(descriptions.strip())


    # #scraping data for the number of likes
    likes_tags = soup_doc.find_all("div", {"class":"head_right"})
    for like in likes_tags:
        likes = like.find_all("span")[1].text.strip()
        number_likes.append(likes)


    # #scraping data for the urls of places
    url_tags = soup_doc.find_all("div", {"class":"head_left"})
    for links in url_tags:
        places_url = links.find_all("a")
        places_urls = base_url + places_url[0]["href"]
        url.append(places_urls)

    pages += 1

else:
    time.sleep(3)
    print("Scraping completed...\n")
    print("Storing extracted information in a csv file...")
    time.sleep(5)
    print("Storing Complete")

    places_dict = dict(
        Places = name_of_place,
        Location = location,
        Category = category,
        Description = description,
        Likes_count = number_likes,
        Links = url
    )

    interesting_sights_in_nigeria = pd.DataFrame(places_dict)
    interesting_sights_in_nigeria.to_csv("Sights_in_Nigeria.csv", index= False)